In [ ]:
import os
import time
import pandas as pd
from ROOT import TChain
from delphes_reader import DelphesLoader
from etaprogress.progress import ProgressBar
from delphes_reader.clasificator import get_met
from delphes_reader.clasificator import get_jets
from delphes_reader.clasificator import get_good_jets
from delphes_reader.clasificator import get_good_leptons
from delphes_reader.clasificator import get_unified
from delphes_reader.root_analysis import get_kinematics_row

def eventSelection(signal,folder_out):
    start_time = time.time()
    #counter
    def count_event(cut_dict,name):
        if( cut_dict.get(name,-1)==-1):
            cut_dict.update({name:0})
        cut_dict[name]+=1
    #Get .root files list
    signal=DelphesLoader(signal)
    print("="*70, flush=True)
    nEvents=signal.nevents
    nTrees=len(signal.Forest)
    cutflow={}
    
    #Set Progress Bar
    bar = ProgressBar(nTrees, max_width=70)
    printEachPercent = 25
    bar.numerator = 0
    print(signal.name, flush=True)
    print(bar, flush=True)
    print("="*70, flush=True)
    nSplits = int(100/printEachPercent)
    last_printed = 0
    
    #Declare dict to save results into csv files 
    results_dict={
        #hadronic selection
        "b_tau_tau_hadronic_zero_jets":[],
        "b_tau_tau_hadronic_(al)one_jet":[],
        "b_b_tau_tau_hadronic":[],
        #semileptonic selection
        "b_tau_tau_semileptonic_zero_jets":[],
        "b_tau_tau_semileptonic_(al)one_jet":[],
        "b_b_tau_tau_semileptonic":[],
        #leptonic selection
        "b_tau_tau_leptonic_zero_jets":[],
        "b_tau_tau_leptonic_(al)one_jet":[],
        "b_b_tau_tau_leptonic":[]
    }
    
    #Declare cut flow dict for each selection
    for key in results_dict.keys():
        cutflow.update({key:{}})
        cutflow[key].update({"xs":signal.xs})
    
    for i,path_root in enumerate(signal.Forest):
        #if i > 0 : break
    
        tree=TChain("Delphes;1")
        tree.Add(path_root)

        for event in tree:
            for key in results_dict.keys():
                count_event(cutflow[key],"All_events")
                
            
            jets = get_good_jets(event)
            l_jets=jets['l_jet']
            b_jets=jets['b_jet']
            tau_jets=jets['tau_jet']
            
            MET=get_met(event)
            row={
                "MET(GeV)":MET.pt(),
                "#phi_{MET}":MET.phi(),
                "light_jets_multiplicity": len(l_jets)
            }
            
            
            if (len(tau_jets)>=2):
                tau_l=tau_jets[0]
                tau_s=tau_jets[1]
                Charges = tau_l.Charge*tau_s.Charge
                row[f"Q_{tau_l.Name}Q_{tau_s.Name}"]=Charges
                
                ##Full-Hadronic selection
                for key in results_dict.keys(): 
                    if "_hadronic" in key: 
                        count_event(
                            cutflow[key],
                            "At least two good hadronic taus"
                        )
                ##
                if (len(b_jets)==1):
                    b=b_jets[0]
                    count_event(
                        cutflow["b_tau_tau_hadronic_zero_jets"],
                        "Exactly one b jet"
                    )
                    count_event(
                        cutflow["b_tau_tau_hadronic_(al)one_jet"],
                        "Exactly one b jet"
                    )
                    
                    if (len(l_jets)==0):
                        count_event(
                            cutflow["b_tau_tau_hadronic_zero_jets"],
                            "Exactly zero good light jet"
                        )
                        row.update(
                            get_kinematics_row(
                                b,
                                tau_l,
                                tau_s
                            )
                        )
                        results_dict["b_tau_tau_hadronic_zero_jets"]+=[row]
                    else:
                        count_event(
                            cutflow["b_tau_tau_hadronic_(al)one_jet"],
                            "At least one light jet"
                        )
                        
                        j=l_jets[0]
                        row.update(
                            get_kinematics_row(
                                b,
                                tau_l,
                                tau_s,
                                j
                            )   
                        )
                        results_dict["b_tau_tau_hadronic_(al)one_jet"]+=[row]
                elif(len(b_jets)>=2):
                    count_event(
                        cutflow["b_b_tau_tau_hadronic"],
                        "At least two b jet"
                    )
                    
                    b_l=b_jets[0]
                    b_s=b_jets[1]
                    
                    row.update(
                        get_kinematics_row(
                            b_l,b_s,
                            tau_l,tau_s
                        )
                    )
                    results_dict["b_b_tau_tau_hadronic"]+=[row]
            elif (len(tau_jets)==0):
                ##Full-leptonic selection
                leptons=get_good_leptons(event)
                
                for key in results_dict.keys(): 
                    if "_leptonic" in key: 
                        count_event(
                            cutflow[key],
                            "Exactly zero good hadronic taus"
                        )
                ##
                if not( len(leptons) >= 2 ): continue
                lep_l=leptons[0]
                lep_s=leptons[1]
                Charges = lep_l.Charge*lep_s.Charge
                row[f"Q_{lep_l.Name}Q_{lep_s.Name}"]=Charges
                for key in results_dict.keys(): 
                    if "_leptonic" in key: 
                        count_event(
                            cutflow[key],
                            "At Least two good leptons"
                        )
                ##
                ##
                if (len(b_jets)==1):
                    b=b_jets[0]
                    count_event(
                        cutflow["b_tau_tau_leptonic_zero_jets"],
                        "Exactly one b jet"
                    )
                    count_event(
                        cutflow["b_tau_tau_leptonic_(al)one_jet"],
                        "Exactly one b jet"
                    )
                    
                    if (len(l_jets)==0):
                        count_event(
                            cutflow["b_tau_tau_leptonic_zero_jets"],
                            "Exactly zero good light jet"
                        )
                        row.update(
                            get_kinematics_row(
                                b,
                                lep_l,
                                lep_s
                            )
                        )
                        results_dict["b_tau_tau_leptonic_zero_jets"]+=[row]
                    else:
                        count_event(
                            cutflow["b_tau_tau_leptonic_(al)one_jet"],
                            "At least one good light jet"
                        )
                        
                        j=l_jets[0]
                        row.update(
                            get_kinematics_row(
                                b,
                                lep_l,
                                lep_s,
                                j
                            )   
                        )
                        results_dict["b_tau_tau_leptonic_(al)one_jet"]+=[row]
                elif(len(b_jets)>=2):
                    count_event(
                        cutflow["b_b_tau_tau_leptonic"],
                        "At least two good b jets"
                    )
                    
                    b_l=b_jets[0]
                    b_s=b_jets[1]
                    
                    row.update(
                        get_kinematics_row(
                            b_l,b_s,
                            lep_l,lep_s
                        )
                    )
                    results_dict["b_b_tau_tau_leptonic"]+=[row]
            elif (len(tau_jets)==1): 
                ##Semi-leptonic selection
                leptons=get_good_leptons(event)
                
                for key in results_dict.keys(): 
                    if "_semileptonic" in key: 
                        count_event(
                            cutflow[key],
                            "Exactly one good hadronic tau"
                        )
                ##
                if not( len(leptons) >=1 ): continue
                lep=leptons[0]
                tau=tau_jets[0]
                Charges = lep.Charge*tau.Charge
                row[f"Q_{lep.Name}Q_{tau.Name}"]=Charges
                for key in results_dict.keys(): 
                    if "_semileptonic" in key: 
                        count_event(
                            cutflow[key],
                            "At Least one good leptons"
                        )
                ##
                ##
                if (len(b_jets)==1):
                    b=b_jets[0]
                    count_event(
                        cutflow["b_tau_tau_semileptonic_zero_jets"],
                        "Exactly one b jet"
                    )
                    count_event(
                        cutflow["b_tau_tau_semileptonic_(al)one_jet"],
                        "Exactly one b jet"
                    )
                    
                    if (len(l_jets)==0):
                        count_event(
                            cutflow["b_tau_tau_semileptonic_zero_jets"],
                            "Exactly zero good light jet"
                        )
                        row.update(
                            get_kinematics_row(
                                b,
                                lep,
                                tau
                            )
                        )
                        results_dict["b_tau_tau_semileptonic_zero_jets"]+=[row]
                    else:
                        count_event(
                            cutflow["b_tau_tau_semileptonic_(al)one_jet"],
                            "At least one good light jet"
                        )
                        
                        j=l_jets[0]
                        row.update(
                            get_kinematics_row(
                                b,
                                lep,
                                tau,
                                j
                            )   
                        )
                        results_dict["b_tau_tau_semileptonic_(al)one_jet"]+=[row]
                elif(len(b_jets)>=2):
                    count_event(
                        cutflow["b_b_tau_tau_semileptonic"],
                        "At least two good b jets"
                    )
                    
                    b_l=b_jets[0]
                    b_s=b_jets[1]
                    
                    row.update(
                        get_kinematics_row(
                            b_l,b_s,
                            lep,tau
                        )
                    )
                    results_dict["b_b_tau_tau_semileptonic"]+=[row]
            ###
        if (int(nSplits*(i)/nTrees)!=last_printed) or (i+1 == nTrees):
            step=i+1
            last_printed=int(nSplits*i/nTrees)
            bar.numerator=step
            print("="*70, flush=True)
            print(signal.name)
            print(bar)
            print("="*70, flush=True)
    for key in results_dict.keys():
        results_dict[key]=pd.DataFrame.from_records(results_dict[key])
        results_dict[key].to_csv(
            os.path.join(folder_out,f"{signal.name}_{key}.csv"), 
            index=False
        )

    elapsed=(time.time() - start_time)/3600.
    print("%"*70, flush=True)
    print("%"*70, flush=True)
    print(signal.name,"done!")
    print(f"time elapsed: {elapsed} hours.")
    print("%"*70, flush=True)
    print("%"*70, flush=True)
    return ( signal.name , cutflow )

In [1]:
import os
import time
from multiprocessing import Pool
from delphes_reader import Quiet

signal_list=["Tau_LQ_Merged_0250","Tau_LQ_0250","LQ_LQ_0250"]
signal_list+=["Tau_LQ_Merged_0500","Tau_LQ_0500","LQ_LQ_0500"]
signal_list+=["Tau_LQ_Merged_0750","Tau_LQ_0750","LQ_LQ_0750"]
signal_list+=["Tau_LQ_Merged_1000","Tau_LQ_1000","LQ_LQ_1000"]
signal_list+=["Tau_LQ_Merged_1250","Tau_LQ_1250","LQ_LQ_1250"]
signal_list+=["Tau_LQ_Merged_1500","Tau_LQ_1500","LQ_LQ_1500"]
signal_list+=["Tau_LQ_Merged_1750","Tau_LQ_1750","LQ_LQ_1750"]
signal_list+=["Tau_LQ_Merged_2000","Tau_LQ_2000","LQ_LQ_2000"]
signal_list+=["Tau_LQ_Merged_2250","Tau_LQ_2250","LQ_LQ_2250"]
signal_list+=["Tau_LQ_Merged_2500","Tau_LQ_2500","LQ_LQ_2500"]

top_list=["ttbar","stop"]
v_jets_lists=["w_jets","z_jets"]
diboson_list=["ww","wz","zz"]

signal_list=top_list+v_jets_lists+diboson_list+signal_list


folder_out=os.path.join(
    os.getcwd(),
    "Leptoquarks_searches",
    "03_delphes_analysis",
    "csv_files"
)

!rm -rf {folder_out}
!mkdir -p {folder_out}

def f(name):
    y=eventSelection(name,folder_out)
    return y 

with Quiet():
    start_time = time.time()
    with Pool(6) as p:  
        goodEvents_dict=dict(p.map(f,signal_list))
    elapsed=(time.time() - start_time)/3600
    print(f"time elapsed: {elapsed} hours.")



Welcome to JupyROOT 6.22/06
Tau_LQ_Merged_0500 imported!
7 trees and 348436 events have been loaded from
/Madgraph_Simulations/LeptoQuarks/tau_lq_merg/tau_lq_merg_m_500

Tau_LQ_Merged_0500
  0% (0/7) [                                             ] eta --:-- /
Tau_LQ_0250 imported!
7 trees and 350000 events have been loaded from
/Madgraph_Simulations/LeptoQuarks/tau_lq/tau_lq_m_250

Tau_LQ_0250
  0% (0/7) [                                             ] eta --:-- /
zz imported!
200 trees and 10000000 events have been loaded from
/Madgraph_Simulations/SM_Backgrounds/zz/

zz
  0% (  0/200) [                                         ] eta --:-- /
w_jets imported!
599 trees and 20942823 events have been loaded from
/Madgraph_Simulations/SM_Backgrounds/w_jets/

w_jets
  0% (  0/599) [                                         ] eta --:-- /
ww imported!
250 trees and 12500000 events have been loaded from
/Madgraph_Simulations/SM_Backgrounds/ww/

ww
  0% (  0/250) [                                

ValueError: All arrays must be of the same length

In [26]:
print("done")

done


In [27]:
import pandas as pd
def get_efficiencies_df(cutflow_dict,folder_out):
    dataframe_dict={}
    for name in next(iter(cutflow_dict.items()))[1].keys():
        #cut names
        results_dict={
            "cut":
            [a[0] for a in next(iter(cutflow_dict.items()))[1][name].items()]
        }
        results_dict["cut"].append("nEvents (137/fb)")
        for key in cutflow_dict.keys():
            if ("_leptonic" in name) and ("w_jets"in key): continue
            signal_dict=cutflow_dict[key][name]
            results_dict.update({key:[]})
            last_cut=1.0
            for cut_key in signal_dict.keys():
                if "xs" == cut_key : 
                    results_dict[key].append(signal_dict[cut_key])
                else :
                    last_cut=signal_dict[cut_key]/signal_dict['All_events']
                    results_dict[key].append(last_cut)
            results_dict[key].append(last_cut*float(signal_dict["xs"])*137.*1000.)
            
        df=pd.DataFrame.from_dict(results_dict)
        df.to_csv(
            os.path.join(folder_out,f"Cutflow_{name}.csv"), 
            index=False
        )
        dataframe_dict.update(
            {name:df}
        )
    for selection in dataframe_dict.keys():
        df = dataframe_dict[selection]
    return dataframe_dict

In [28]:
efficiencies=get_efficiencies_df(goodEvents_dict,folder_out)
for selection in efficiencies.keys():
    print("%"*80)
    print(selection)
    print(efficiencies[selection])

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
b_tau_tau_hadronic_zero_jets
                               cut         ttbar         stop        w_jets  \
0                               xs  5.046960e+02       241.17  1.465483e+05   
1                       All_events           1.0          1.0           1.0   
2  At least two good hadronic taus      0.003147      0.00159      0.000138   
3                Exactly one b jet      0.001517     0.000609      0.000005   
4      Exactly zero good light jet       0.00014     0.000113      0.000002   
5                 nEvents (137/fb)   9657.270158  3735.805517  36429.207743   

         z_jets            ww            wz            zz Tau_LQ_Merged_0250  \
0  4.302927e+04  6.550736e+01  2.469184e+01  9.562416e+00       1.396351e+02   
1           1.0           1.0           1.0           1.0                1.0   
2      0.001355      0.001132       0.00244      0.003927           0.123865   
3      0.000018 